In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/redditresampled/rdResampled.csv


In [3]:
import pandas as pd 
df=pd.read_csv('/kaggle/input/redditresampled/rdResampled.csv')

In [4]:
df.head()

,Unnamed: 0,title_post,Subreddit
0,0,The end of the road. I've never posted on redd...,depression
1,1,I can’t write comments. I think I should not c...,depression
2,2,Attempted hanging myself two years ago. So nea...,depression
3,3,"25/ f started deliberate self injury. Hi all, ...",depression
4,4,Venting I guess. I've been crying in my high s...,depression


In [5]:
df=df.drop(['Unnamed: 0'],axis=1)
df.head()

,title_post,Subreddit
0,The end of the road. I've never posted on redd...,depression
1,I can’t write comments. I think I should not c...,depression
2,Attempted hanging myself two years ago. So nea...,depression
3,"25/ f started deliberate self injury. Hi all, ...",depression
4,Venting I guess. I've been crying in my high s...,depression


In [6]:
df['Subreddit'].value_counts()

depression       35000
Anxiety          35000
bipolar          35000
BPD              35000
schizophrenia    35000
autism           35000
Name: Subreddit, dtype: int64

In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
X = df['title_post']
y = df['Subreddit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print(len(X_train),len(X_test))

168000 42000


In [11]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [12]:
train_encoded = tokenizer.batch_encode_plus(X_train.tolist(), truncation=True, padding=True, max_length=256, return_tensors='tf')
test_encoded = tokenizer.batch_encode_plus(X_test.tolist(), truncation=True, padding=True, max_length=256, return_tensors='tf')


In [13]:
num_examples = train_encoded['input_ids'].shape[0]
print("Number of examples:", num_examples)

Number of examples: 168000


In [14]:
example_input_ids = train_encoded['input_ids'][1]
print(example_input_ids)

tf.Tensor(
[    0  1106    38  2341     8  1524    19 35643    31  1817    40    51
    28   441     7   146   127  1508   555   101    89  1508   116     4
   318    38  2341     8  1524    19 35643    31  1817    40    51    28
   441     7   146   127  1508   555   101    89  1508   116     2     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1    

In [15]:
label_mapping = {label: i for i, label in enumerate(df['Subreddit'].unique())}
y_train = y_train.map(label_mapping)
y_test = y_test.map(label_mapping)

In [16]:
label_mapping

{'depression': 0,
 'Anxiety': 1,
 'bipolar': 2,
 'BPD': 3,
 'schizophrenia': 4,
 'autism': 5}

In [17]:
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=6)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [19]:
loss

In [20]:
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 595206    
 ficationHead)                                                   
                                                                 
Total params: 124,650,246
Trainable params: 124,650,246
Non-trainable params: 0
_________________________________________________________________


In [21]:
import numpy as np

# Convert train_encoded and test_encoded to numpy arrays
train_inputs = train_encoded['input_ids'].numpy()
train_attention_mask = train_encoded['attention_mask'].numpy()
test_inputs = test_encoded['input_ids'].numpy()
test_attention_mask = test_encoded['attention_mask'].numpy()

# Convert y_train to numpy array
y_train = np.array(y_train)

In [22]:
print(len(y_train), type(y_train))

168000 <class 'numpy.ndarray'>


In [23]:
print(len(y_test), type(y_test))

42000 <class 'pandas.core.series.Series'>


In [24]:
batch_size = 16
history = model.fit(
    x=[train_inputs, train_attention_mask],
    y=y_train,
    batch_size=batch_size,
    epochs=3,
    validation_data=([test_inputs, test_attention_mask], y_test)
)

Epoch 1/3
10500/10500 [==============================] - 5487s 519ms/step - loss: 0.5897 - accuracy: 0.7958 - val_loss: 0.4908 - val_accuracy: 0.8300
Epoch 2/3
10500/10500 [==============================] - 5439s 518ms/step - loss: 0.4361 - accuracy: 0.8493 - val_loss: 0.4568 - val_accuracy: 0.8435
Epoch 3/3
10500/10500 [==============================] - 5437s 518ms/step - loss: 0.3579 - accuracy: 0.8758 - val_loss: 0.4507 - val_accuracy: 0.8514


In [25]:
# Saving model method 2
model.save_weights('/kaggle/working/RoBERTa_model_weights.h5')

In [27]:
input_ids = test_encoded['input_ids'].numpy()
attention_mask = test_encoded['attention_mask'].numpy()

predictions = model.predict([input_ids,attention_mask])

1313/1313 [==============================] - 420s 318ms/step


In [28]:
label_mapping_reverse = {0: 'depression', 1: 'Anxiety', 2: 'bipolar', 3: 'BPD', 4: 'schizophrenia', 5: 'autism'}

predicted_labels = tf.argmax(predictions.logits, axis=1)
predicted_class_names = [label_mapping_reverse[label.numpy()] for label in predicted_labels]

In [29]:
predicted_class_names[:5]

['autism', 'schizophrenia', 'Anxiety', 'Anxiety', 'schizophrenia']

In [30]:
y_test[:5]

194949    5
161875    4
61912     1
35966     1
143596    4
Name: Subreddit, dtype: int64

In [31]:
print(type(predicted_class_names))

<class 'list'>


In [34]:
# Convert the numerical labels in y_test to class names
y_test_strings = [label_mapping_reverse[label] for label in y_test]

In [35]:
from sklearn.metrics import classification_report

# Calculate classification report
classification_metrics = classification_report(y_test_strings, predicted_class_names)
print(classification_metrics)

               precision    recall  f1-score   support

      Anxiety       0.86      0.83      0.85      6883
          BPD       0.82      0.77      0.80      6926
       autism       0.97      0.99      0.98      7069
      bipolar       0.82      0.78      0.80      7108
   depression       0.74      0.84      0.79      6950
schizophrenia       0.90      0.89      0.89      7064

     accuracy                           0.85     42000
    macro avg       0.85      0.85      0.85     42000
 weighted avg       0.85      0.85      0.85     42000

